# 模板函数与模板类 - 实践篇

本notebook通过实际代码帮助你理解C++模板的核心概念。

**学习目标：**
- 编写函数模板和类模板
- 理解模板参数推导
- 使用非类型模板参数
- 掌握变参模板基础


## 环境说明

本notebook使用 `%%writefile` 魔法命令将代码写入文件，然后使用 `g++` 编译运行。

确保你的环境支持 C++17：
```bash
g++ --version  # 需要 GCC 7.0+
```


## 1. 函数模板基础

让我们从最简单的函数模板开始：实现一个通用的 `swap` 和 `max` 函数。


In [ ]:
%%writefile template_basics.cpp
#include <iostream>
#include <string>

// 函数模板：交换两个值
template <typename T>
void mySwap(T& a, T& b) {
    T temp = a;
    a = b;
    b = temp;
}

// 函数模板：返回较大值
template <typename T>
T myMax(T a, T b) {
    return (a > b) ? a : b;
}

// 多类型参数的函数模板
template <typename T, typename U>
auto add(T a, U b) {
    return a + b;  // C++14: 返回类型自动推导
}

int main() {
    std::cout << "==========================================" << std::endl;
    std::cout << "           函数模板基础演示" << std::endl;
    std::cout << "==========================================\n" << std::endl;
    
    // 测试 mySwap
    std::cout << "【mySwap 测试】" << std::endl;
    int x = 10, y = 20;
    std::cout << "交换前: x = " << x << ", y = " << y << std::endl;
    mySwap(x, y);  // 自动推导 T = int
    std::cout << "交换后: x = " << x << ", y = " << y << std::endl;
    
    std::string s1 = "Hello", s2 = "World";
    std::cout << "\n交换前: s1 = " << s1 << ", s2 = " << s2 << std::endl;
    mySwap(s1, s2);  // 自动推导 T = std::string
    std::cout << "交换后: s1 = " << s1 << ", s2 = " << s2 << std::endl;
    
    // 测试 myMax
    std::cout << "\n【myMax 测试】" << std::endl;
    std::cout << "myMax(10, 20) = " << myMax(10, 20) << std::endl;
    std::cout << "myMax(3.14, 2.71) = " << myMax(3.14, 2.71) << std::endl;
    std::cout << "myMax('a', 'z') = " << myMax('a', 'z') << std::endl;
    
    // 显式指定模板参数
    std::cout << "myMax<double>(10, 3.14) = " << myMax<double>(10, 3.14) << std::endl;
    
    // 测试 add（多类型参数）
    std::cout << "\n【add 测试 - 多类型参数】" << std::endl;
    std::cout << "add(1, 2.5) = " << add(1, 2.5) << std::endl;
    std::cout << "add(3.14f, 10) = " << add(3.14f, 10) << std::endl;
    
    return 0;
}


In [ ]:
!g++ -std=c++17 -o template_basics template_basics.cpp && ./template_basics


## 2. 类模板基础

实现一个简单的泛型容器类 `Box` 和栈 `Stack`。


In [ ]:
%%writefile class_template.cpp
#include <iostream>
#include <vector>
#include <string>
#include <stdexcept>

// 简单的盒子类模板
template <typename T>
class Box {
private:
    T content;
    
public:
    Box(T value) : content(value) {}
    
    T get() const { return content; }
    void set(T value) { content = value; }
    
    // 成员模板：允许与不同类型的Box比较
    template <typename U>
    bool isLargerThan(const Box<U>& other) const {
        return content > other.get();
    }
};

// 栈类模板
template <typename T>
class Stack {
private:
    std::vector<T> data;
    
public:
    void push(const T& value) { data.push_back(value); }
    
    T pop() {
        if (data.empty()) throw std::runtime_error("Stack is empty!");
        T top = data.back();
        data.pop_back();
        return top;
    }
    
    bool empty() const { return data.empty(); }
    size_t size() const { return data.size(); }
};

int main() {
    std::cout << "==========================================" << std::endl;
    std::cout << "           类模板基础演示" << std::endl;
    std::cout << "==========================================\n" << std::endl;
    
    // 测试 Box
    std::cout << "【Box 测试】" << std::endl;
    Box<int> intBox(42);
    Box<double> doubleBox(3.14);
    Box<std::string> strBox("Hello Template");
    
    std::cout << "intBox: " << intBox.get() << std::endl;
    std::cout << "doubleBox: " << doubleBox.get() << std::endl;
    std::cout << "strBox: " << strBox.get() << std::endl;
    
    std::cout << "\nintBox(42) > doubleBox(3.14)? " 
              << (intBox.isLargerThan(doubleBox) ? "Yes" : "No") << std::endl;
    
    // 测试 Stack
    std::cout << "\n【Stack 测试】" << std::endl;
    Stack<int> intStack;
    
    std::cout << "Push: 10, 20, 30" << std::endl;
    intStack.push(10);
    intStack.push(20);
    intStack.push(30);
    
    std::cout << "Stack size: " << intStack.size() << std::endl;
    std::cout << "Pop elements: ";
    while (!intStack.empty()) {
        std::cout << intStack.pop() << " ";
    }
    std::cout << std::endl;
    
    return 0;
}


In [ ]:
!g++ -std=c++17 -o class_template class_template.cpp && ./class_template


## 3. 非类型模板参数

模板参数不仅可以是类型，还可以是编译期常量。这在FlashAttention中被大量使用。


In [ ]:
%%writefile nontype_template.cpp
#include <iostream>

// 固定大小的数组类模板
template <typename T, int Size>
class FixedArray {
private:
    T data[Size];  // 编译期确定大小
    
public:
    constexpr int size() const { return Size; }
    
    T& operator[](int index) { return data[index]; }
    const T& operator[](int index) const { return data[index]; }
    
    void fill(T value) {
        for (int i = 0; i < Size; i++) data[i] = value;
    }
    
    void print() const {
        std::cout << "[";
        for (int i = 0; i < Size; i++) {
            std::cout << data[i];
            if (i < Size - 1) std::cout << ", ";
        }
        std::cout << "]" << std::endl;
    }
};

// 编译期阶乘计算（模板元编程经典例子）
template <int N>
struct Factorial {
    static constexpr int value = N * Factorial<N-1>::value;
};

template <>
struct Factorial<0> {
    static constexpr int value = 1;
};

// 模拟FlashAttention中的Kernel Traits
template <int BlockM, int BlockN, int HeadDim>
struct KernelTraits {
    static constexpr int kBlockM = BlockM;
    static constexpr int kBlockN = BlockN;
    static constexpr int kHeadDim = HeadDim;
    static constexpr int kBlockElements = BlockM * BlockN;
    
    static void printConfig() {
        std::cout << "KernelTraits: BlockM=" << kBlockM 
                  << ", BlockN=" << kBlockN 
                  << ", HeadDim=" << kHeadDim 
                  << ", Elements=" << kBlockElements << std::endl;
    }
};

int main() {
    std::cout << "==========================================" << std::endl;
    std::cout << "         非类型模板参数演示" << std::endl;
    std::cout << "==========================================\n" << std::endl;
    
    // 测试 FixedArray
    std::cout << "【FixedArray 测试】" << std::endl;
    FixedArray<int, 5> arr;
    arr.fill(0);
    arr[0] = 1; arr[1] = 2; arr[2] = 3;
    std::cout << "arr (size=" << arr.size() << "): ";
    arr.print();
    
    // 编译期阶乘
    std::cout << "\n【编译期阶乘计算】" << std::endl;
    std::cout << "Factorial<5>::value = " << Factorial<5>::value << std::endl;
    std::cout << "Factorial<10>::value = " << Factorial<10>::value << std::endl;
    
    constexpr int fact7 = Factorial<7>::value;  // 编译期计算
    std::cout << "Factorial<7>::value (constexpr) = " << fact7 << std::endl;
    
    // 模拟FlashAttention的KernelTraits
    std::cout << "\n【KernelTraits 模拟】" << std::endl;
    KernelTraits<128, 64, 64>::printConfig();
    KernelTraits<64, 128, 128>::printConfig();
    
    return 0;
}


In [ ]:
!g++ -std=c++17 -o nontype_template nontype_template.cpp && ./nontype_template


## 4. 变参模板

变参模板允许接受任意数量的参数，是实现通用print函数、元组等的基础。


In [ ]:
%%writefile variadic_template.cpp
#include <iostream>
#include <string>

// ==================== 递归展开方式 ====================

// 基础情况：单个参数
template <typename T>
void print(T value) {
    std::cout << value << std::endl;
}

// 递归情况：处理第一个参数，然后递归处理剩余参数
template <typename T, typename... Args>
void print(T first, Args... rest) {
    std::cout << first << " ";
    print(rest...);  // 递归调用
}

// ==================== C++17 折叠表达式 ====================

// 使用折叠表达式求和
template <typename... Args>
auto sum(Args... args) {
    return (args + ...);  // 一元右折叠
}

// 使用折叠表达式打印
template <typename... Args>
void printFold(Args... args) {
    ((std::cout << args << " "), ...);  // 逗号折叠
    std::cout << std::endl;
}

// 检查所有参数是否都为true
template <typename... Args>
bool allTrue(Args... args) {
    return (args && ...);
}

// 检查是否有任一参数为true
template <typename... Args>
bool anyTrue(Args... args) {
    return (args || ...);
}

// 参数包大小
template <typename... Args>
void showPackInfo(Args... args) {
    std::cout << "参数包包含 " << sizeof...(Args) << " 个类型" << std::endl;
    std::cout << "参数包包含 " << sizeof...(args) << " 个值" << std::endl;
}

int main() {
    std::cout << "==========================================" << std::endl;
    std::cout << "           变参模板演示" << std::endl;
    std::cout << "==========================================\n" << std::endl;
    
    // 递归展开
    std::cout << "【递归展开方式】" << std::endl;
    std::cout << "print(1, 2.5, \"hello\", 'c'): ";
    print(1, 2.5, "hello", 'c');
    
    // 折叠表达式
    std::cout << "\n【C++17 折叠表达式】" << std::endl;
    std::cout << "sum(1, 2, 3, 4, 5) = " << sum(1, 2, 3, 4, 5) << std::endl;
    std::cout << "sum(1.1, 2.2, 3.3) = " << sum(1.1, 2.2, 3.3) << std::endl;
    
    std::cout << "\nprintFold(\"Hello\", 42, 3.14): ";
    printFold("Hello", 42, 3.14);
    
    // 逻辑折叠
    std::cout << "\n【逻辑折叠】" << std::endl;
    std::cout << "allTrue(true, true, true) = " << allTrue(true, true, true) << std::endl;
    std::cout << "allTrue(true, false, true) = " << allTrue(true, false, true) << std::endl;
    std::cout << "anyTrue(false, false, true) = " << anyTrue(false, false, true) << std::endl;
    
    // 参数包信息
    std::cout << "\n【参数包信息】" << std::endl;
    showPackInfo(1, 2.0, "three", 'f');
    
    return 0;
}


In [ ]:
!g++ -std=c++17 -o variadic_template variadic_template.cpp && ./variadic_template


## 清理临时文件


In [ ]:
!rm -f template_basics.cpp template_basics class_template.cpp class_template nontype_template.cpp nontype_template variadic_template.cpp variadic_template


## 总结

通过本notebook，你应该理解了：

1. **函数模板**
   - 使用 `template <typename T>` 定义
   - 编译器可以自动推导类型参数
   - 可以显式指定类型参数

2. **类模板**
   - 使用 `template <typename T>` 定义
   - 使用时需要指定类型参数（C++17前）
   - 可以包含成员模板

3. **非类型模板参数**
   - 允许使用编译期常量作为模板参数
   - FlashAttention用此定义 BlockM, BlockN 等

4. **变参模板**
   - 接受任意数量的参数
   - 使用递归或折叠表达式展开

## 练习

1. 实现一个模板函数 `clamp(value, min, max)`，将值限制在[min, max]范围内
2. 实现一个 `Pair<T, U>` 类模板，类似 `std::pair`
3. 使用变参模板实现一个 `makeArray` 函数，返回 `std::array`
